In [1]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA

In [2]:
# Step2: Read the data from train.csv
df_train = pd.read_csv("E:\\DATASCIENCE\PROJECTS\\train\\train.csv")

In [3]:
df_train.shape

(4209, 378)

In [4]:
# print few rows and see how the data looks like 
df_train

,ID,y,X0,X1,X2,X3,X4,X5,X6,X8,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,0,130.81,k,v,at,a,d,u,j,o,...,0,0,1,0,0,0,0,0,0,0
1,6,88.53,k,t,av,e,d,y,l,o,...,1,0,0,0,0,0,0,0,0,0
2,7,76.26,az,w,n,c,d,x,j,x,...,0,0,0,0,0,0,1,0,0,0
3,9,80.62,az,t,n,f,d,x,l,e,...,0,0,0,0,0,0,0,0,0,0
4,13,78.02,az,v,n,f,d,h,d,n,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4204,8405,107.39,ak,s,as,c,d,aa,d,q,...,1,0,0,0,0,0,0,0,0,0
4205,8406,108.77,j,o,t,d,d,aa,h,h,...,0,1,0,0,0,0,0,0,0,0
4206,8412,109.22,ak,v,r,a,d,aa,g,e,...,0,0,1,0,0,0,0,0,0,0
4207,8415,87.48,al,r,e,f,d,aa,l,u,...,0,0,0,0,0,0,0,0,0,0


In [5]:
# Step3: Collect the Y values into an array
# seperate the y from the data as we will use this to learn as 
# the prediction output
y_train= df_train['y'].value_counts()

In [6]:
# Step4: Understand the data types we have
# iterate through all the columns which has X in the name of the column
cols = [c for c in df_train.columns if 'X' in c]
print('Number of features: {}'.format(len(cols)))

Number of features: 376


In [7]:
print('Feature types:')
df_train[cols].dtypes.value_counts()

Feature types:


int64     368
object      8
dtype: int64

In [8]:
# Step5: Count the data in each of the columns
counts = [[], [], []]
for c in cols:
    typ = df_train[c].dtype
    uniq = len(np.unique(df_train[c]))
    if uniq == 1:
        counts[0].append(c)
    elif uniq == 2 and typ == np.int64:
        counts[1].append(c)
    else:
        counts[2].append(c)

In [9]:
print('Constant features: {} Binary features: {} Categorical features: {}\n'
      .format(*[len(c) for c in counts]))
print('Constant features:', counts[0])
print('Categorical features:', counts[2])

Constant features: 12 Binary features: 356 Categorical features: 8

Constant features: ['X11', 'X93', 'X107', 'X233', 'X235', 'X268', 'X289', 'X290', 'X293', 'X297', 'X330', 'X347']
Categorical features: ['X0', 'X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X8']


In [10]:
# Step6: Read the test.csv data
df_test = pd.read_csv("E:\\DATASCIENCE\PROJECTS\\test\\test.csv")

In [11]:
df_test.head()

,ID,X0,X1,X2,X3,X4,X5,X6,X8,X10,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,1,az,v,n,f,d,t,a,w,0,...,0,0,0,1,0,0,0,0,0,0
1,2,t,b,ai,a,d,b,g,y,0,...,0,0,1,0,0,0,0,0,0,0
2,3,az,v,as,f,d,a,j,j,0,...,0,0,0,1,0,0,0,0,0,0
3,4,az,l,n,f,d,z,l,n,0,...,0,0,0,1,0,0,0,0,0,0
4,5,w,s,as,c,d,y,i,m,0,...,1,0,0,0,0,0,0,0,0,0


In [12]:
# remove columns ID and Y from the data as they are not used for learning
usable_columns = list(set(df_train.columns) - set(['ID', 'y']))
y_train = df_train['y'].values
id_test = df_test['ID'].values

In [25]:
x_train = df_train[usable_columns]
x_test = df_test[usable_columns]

In [14]:
# Step7: Check for null and unique values for test and train sets
def check_missing_values(df):
    if df.isnull().any().any():
        print("There are missing values in the dataframe")
    else:
        print("There are no missing values in the dataframe")
check_missing_values(x_train)
check_missing_values(x_test)


There are no missing values in the dataframe
There are no missing values in the dataframe


In [15]:
# Step8: If for any column(s), the variance is equal to zero, 
# then you need to remove those variable(s).
# Apply label encoder
for column in usable_columns:
    cardinality = len(np.unique(x_train[column]))
    if cardinality == 1:
        x_train.drop(column, axis=1) # Column with only one 
        # value is useless so we drop it
        x_test.drop(column, axis=1)
    if cardinality > 2: # Column is categorical
        mapper = lambda x: sum([ord(digit) for digit in x])
        x_train[column] = x_train[column].apply(mapper)
        x_test[column] = x_test[column].apply(mapper)
x_train.head()

C:\Users\MITHILESH\AppData\Local\Temp\ipykernel_14520\3305823630.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_train[column] = x_train[column].apply(mapper)
C:\Users\MITHILESH\AppData\Local\Temp\ipykernel_14520\3305823630.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_test[column] = x_test[column].apply(mapper)
C:\Users\MITHILESH\AppData\Local\Temp\ipykernel_14520\3305823630.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row

,X191,X239,X299,X262,X78,X30,X101,X283,X168,X282,...,X207,X21,X318,X322,X368,X140,X23,X260,X371,X296
0,0,0,0,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [16]:
# Step9: Make sure the data is now changed into numericals

print('Feature types:')
x_train[cols].dtypes.value_counts()


Feature types:


int64    376
dtype: int64

In [17]:
# Step10: Perform dimensionality reduction
# Linear dimensionality reduction using Singular Value Decomposition of 
# the data to project it to a lower dimensional space.
n_comp = 12
pca = PCA(n_components=n_comp, random_state=420)
pca2_results_train = pca.fit_transform(x_train)
pca2_results_test = pca.transform(x_test)

In [18]:
pip install xgboost

In [19]:
import xgboost

print(dir(xgboost))

['Booster', 'DMatrix', 'DataIter', 'DeviceQuantileDMatrix', 'RabitTracker', 'XGBClassifier', 'XGBModel', 'XGBRFClassifier', 'XGBRFRegressor', 'XGBRanker', 'XGBRegressor', '__all__', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', '__version__', '_py_version', '_typing', 'build_info', 'callback', 'compat', 'config', 'config_context', 'core', 'cv', 'dask', 'data', 'get_config', 'libpath', 'plot_importance', 'plot_tree', 'plotting', 'rabit', 'set_config', 'sklearn', 'to_graphviz', 'tracker', 'train', 'training']


In [20]:
# Step11: Training using xgboost

import xgboost as xgb
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

x_train, x_valid, y_train, y_valid = train_test_split(pca2_results_train, y_train, test_size=0.2,random_state=4242)

d_train = xgb.DMatrix(x_train, label=y_train)
d_valid = xgb.DMatrix(x_valid, label=y_valid)
#d_test = xgb.DMatrix(x_test)
d_test = xgb.DMatrix(pca2_results_test)

params = {}
params['objective'] = 'reg:linear'
params['eta'] = 0.02
params['max_depth'] = 4

def xgb_r2_score(preds, dtrain):
    labels = dtrain.get_label()
    return 'r2', r2_score(labels, preds)

watchlist = [(d_train, 'train'), (d_valid, 'valid')]

clf = xgb.train(params, d_train, 
                1000, watchlist, early_stopping_rounds=50, 
                feval=xgb_r2_score, maximize=True, verbose_eval=10)

[11:32:05] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.
[0]	train-rmse:99.14834	train-r2:-58.35295	valid-rmse:98.26297	valid-r2:-67.63754
[10]	train-rmse:81.27653	train-r2:-38.88428	valid-rmse:80.36433	valid-r2:-44.91014
[20]	train-rmse:66.71610	train-r2:-25.87403	valid-rmse:65.77334	valid-r2:-29.75260


C:\Users\MITHILESH\anaconda3\lib\site-packages\xgboost\core.py:568: FutureWarning: Pass `evals` as keyword args.  Passing these as positional arguments will be considered as error in future releases.
  warnings.warn(
C:\Users\MITHILESH\anaconda3\lib\site-packages\xgboost\training.py:35: UserWarning: `feval` is deprecated, use `custom_metric` instead.  They have different behavior when custom objective is also used.See https://xgboost.readthedocs.io/en/latest/tutorials/custom_metric_obj.html for details on the `custom_metric`.
  warnings.warn(


[30]	train-rmse:54.86912	train-r2:-17.17722	valid-rmse:53.89136	valid-r2:-19.64525
[40]	train-rmse:45.24709	train-r2:-11.36097	valid-rmse:44.22323	valid-r2:-12.90218
[50]	train-rmse:37.44854	train-r2:-7.46723	valid-rmse:36.37628	valid-r2:-8.40630
[60]	train-rmse:31.14582	train-r2:-4.85694	valid-rmse:30.02269	valid-r2:-5.40739
[70]	train-rmse:26.08414	train-r2:-3.10794	valid-rmse:24.91507	valid-r2:-3.41272
[80]	train-rmse:22.04310	train-r2:-1.93371	valid-rmse:20.83065	valid-r2:-2.08452
[90]	train-rmse:18.84668	train-r2:-1.14457	valid-rmse:17.59606	valid-r2:-1.20096
[100]	train-rmse:16.33345	train-r2:-0.61075	valid-rmse:15.07956	valid-r2:-0.61644
[110]	train-rmse:14.39545	train-r2:-0.25119	valid-rmse:13.14900	valid-r2:-0.22904
[120]	train-rmse:12.93005	train-r2:-0.00942	valid-rmse:11.68809	valid-r2:0.02889
[130]	train-rmse:11.81778	train-r2:0.15677	valid-rmse:10.61773	valid-r2:0.19861
[140]	train-rmse:10.98880	train-r2:0.27093	valid-rmse:9.84613	valid-r2:0.31085
[150]	train-rmse:10.38016

In [21]:
# Step12: Predict your test_df values using xgboost

p_test = clf.predict(d_test)

sub = pd.DataFrame()
sub['ID'] = id_test
sub['y'] = p_test
sub.to_csv('xgb.csv', index=False)

sub.head()

,ID,y
0,1,83.027115
1,2,97.228844
2,3,82.903618
3,4,76.967072
4,5,112.526451


In [22]:
c

'X385'